# Analysis of Imaging Data

## Signal Extraction & Source-Separation

### Import Modules

In [1]:
# FISSA: Signal Extraction & Source-Separation
from AnalysisModules.FissaAnalysis import FissaModule
# Assorted Static Processing Functions
from AnalysisModules.StaticProcessing import smoothTraces_TiffOrg, calculate_dFoF, mergeTraces, detrendTraces

### Instantiation

In [3]:
# Instantiate Fissa Module & Sub-Modules (preparation, experiment (separation), and ProcessedTraces)
Data = FissaModule(data_folder="D:\\M4618_Second_Round\\Encoding", index_file = "D:\\M4618_Second_Round\\Encoding\\suite2p\\plane0\\Debug_Neuronal_Index.csv")
# Initialize
#Data.initializeFissa()
#Data.pruneNonNeuronalROIs()


Adjusting from 1-to-0 indexing


In [4]:
Data.stat = Data.stat[Data.neuronal_index]

In [5]:
#Data.s2p_rois = Data.s2p_rois[Data.neuronal_index]
Data.s2p_rois = [Data.s2p_rois[i] for i in Data.neuronal_index]

In [6]:
Data.iscell = Data.iscell[Data.neuronal_index, :]

In [7]:
Data.initializeFissa()

Initialized Fissa


### Extract Traces

In [8]:
Data.extractTraces()

Extracting traces:   0%|          | 0/6 [00:00<?, ?it/s]

Finished extracting raw signals from 662 ROIs across 6 trials in 2 min, 56 sec.
Passing traces between modules.
Finished module-passing.
Ready for post-processing or source-separation.


### Save Progress

In [9]:
Data.saveFissaPrep()

Saving extracted traces to D:\M4618_Second_Round\Encoding\prepared.npz
Finished Saving Prep


### Pre-Process Traces

In [10]:
Data.ProcessedTraces.smoothed_raw = smoothTraces_TiffOrg(Data.preparation.raw, niter=50, kappa=150, gamma=0.15)[0]

Smoothing:   0%|          | 0/662 [00:00<?, ?it/s]

Organizing By Trial:   0%|          | 0/662 [00:00<?, ?it/s]

### Save Progress

In [11]:
Data.saveProcessedTraces()

Saving Processed Traces...
Finished Saving Processed Traces.


### Use Pre-Processed Traces for Source-Separation

In [12]:
Data.preparation.raw = Data.ProcessedTraces.smoothed_raw.copy()
Data.passPrepToFissa()

### Source-Separate Traces

In [13]:
Data.separateTraces()

Passing prepared traces to fissa.
Initiating fissa source-separation


Separating data:   0%|          | 0/662 [00:00<?, ?it/s]

Finished separating signals from 662 ROIs across 6 trials in 1:12:59.190875
Passing traces between modules.
Finished module-passing.
Ready for further analysis.


### Save Progress

In [14]:
Data.saveFissaSep()

Saving results to D:\M4618_Second_Round\Encoding\separated.npz


In [7]:
Data = FissaModule(data_folder="D:\\M4618_Second_Round\\Encoding", index_file = "D:\\M4618_Second_Round\\Encoding\\suite2p\\plane0\\Debug_Neuronal_Index.csv")
Data.initializeFissa()
# Load
Data.loadFissaPrep()
Data.loadFissaSep()
Data.loadProcessedTraces()

Adjusting from 1-to-0 indexing
Initialized Fissa
Loading Fissa Preparation...
Finished Loading Fissa Prep
Loading Fissa Separation...
Finished Loading Fissa Sep
Loading Processed Traces...
Finished Loading Processed Traces.


### Post-Process Traces

In [3]:
# Calculate Fo/F
Data.ProcessedTraces.dFoF_result = calculate_dFoF(Data.experiment.result, Data.frame_rate, raw=Data.preparation.raw, merge_after=False)
# Condense the ROI Traces for each Trial into a Single Matrix
Data.ProcessedTraces.merged_dFoF_result = mergeTraces(Data.ProcessedTraces.dFoF_result)
# Detrend the Traces
Data.ProcessedTraces.detrended_merged_dFoF_result = detrendTraces(Data.ProcessedTraces.merged_dFoF_result, order=4, plot=False)

Calculating Δf/f0  (using single f0 across all trials using f0 derived from raw traces during calculations)


Calculating Δf/f0:   0%|          | 0/662 [00:00<?, ?it/s]

Merging Traces Across Trials:   0%|          | 0/662 [00:00<?, ?it/s]

Detrending:   0%|          | 0/662 [00:00<?, ?it/s]

### Save Progress

In [4]:
Data.saveProcessedTraces()

Saving Processed Traces...
Finished Saving Processed Traces.


## Infer Firing Rates & Approximate Spikes

### Import Modules

In [8]:
from AnalysisModules.CascadeAnalysis import CascadeModule
from AnalysisModules.StaticProcessing import calculateFiringRate
from AnalysisModules.StaticUtilities import pullModels

### Instantiation

In [9]:
# Pass the Post-Processed Traces
Data.Cascade = CascadeModule(Data.ProcessedTraces.detrended_merged_dFoF_result, Data.frame_rate, model_folder="C:\\ProgramData\\Anaconda3\\envs\\suite2p\\Pretrained_models")

### Model Selection

In [13]:
# Pull Available Models
list_of_models = pullModels(Data.Cascade.model_folder)
# Select Model
Data.Cascade.model_name = list_of_models[21]

You can now check the updated available_models.yaml file for valid model names.
File location: C:\ProgramData\Anaconda3\envs\suite2p\Pretrained_models\available_models.yaml

 List of available models: 

Global_EXC_30Hz_smoothing50ms_asymmetric_window_1_frame
Global_EXC_30Hz_smoothing50ms_asymmetric_window_2_frames
Global_EXC_30Hz_smoothing50ms_asymmetric_window_4_frames
Global_EXC_30Hz_smoothing50ms_asymmetric_window_6_frames
Global_EXC_30Hz_smoothing50ms_asymmetric_window_8_frames
GCaMP6f_mouse_30Hz_smoothing200ms
OGB_zf_pDp_7.5Hz_smoothing200ms
OGB_zf_pDp_7.5Hz_smoothing200ms_causalkernel
Global_EXC_3Hz_smoothing400ms
Global_EXC_3Hz_smoothing400ms_causalkernel
Global_EXC_4.25Hz_smoothing300ms
Global_EXC_4.25Hz_smoothing300ms_high_noise
Global_EXC_4.25Hz_smoothing300ms_causalkernel
Global_EXC_5Hz_smoothing200ms
Global_EXC_5Hz_smoothing200ms_causalkernel
Global_EXC_6Hz_smoothing200ms
Global_EXC_6Hz_smoothing200ms_causalkernel
Global_EXC_7.5Hz_smoothing200ms
Global_EXC_7.5Hz_smoothing20

In [8]:
Data.Cascade.loadSpikeProb(load_path=Data.output_folder)
Data.Cascade.loadProcessedInferences(load_path=Data.output_folder)

Loading Spike Probabilities from Load Path...
Finished Loading Spike Probabilities
Loading Processed Inferences...
Finished Loading Processed Traces.


### Infer Firing Rates

In [14]:
# Infer Spike Probability
Data.Cascade.predictSpikeProb()
# Calculate Firing Rates
Data.Cascade.ProcessedInferences.firing_rates = calculateFiringRate(Data.Cascade.spike_prob, Data.Cascade.frame_rate)


 
The selected model was trained on 18 datasets, with 5 ensembles for each noise level, at a sampling rate of 10Hz, with a resampled ground truth that was smoothed with a Gaussian kernel of a standard deviation of 100 milliseconds. 
 

Loaded model was trained at frame rate 10 Hz
Given argument traces contains 662 neurons and 41703 frames.
Noise levels (mean, std; in standard units): 0.06, 0.04

Predictions for noise level 2:
	... ensemble 0
3371/3371 [==============================] - 15s 4ms/step
	... ensemble 1
3371/3371 [==============================] - 16s 5ms/step
	... ensemble 2
3371/3371 [==============================] - 15s 5ms/step
	... ensemble 3
3371/3371 [==============================] - 15s 5ms/step
	... ensemble 4
3371/3371 [==============================] - 15s 5ms/step

Predictions for noise level 3:
	No neurons for this noise level

Predictions for noise level 4:
	No neurons for this noise level

Predictions for noise level 5:
	No neurons for this noise level

Pre

### Save Progress

In [15]:
Data.Cascade.saveSpikeProb(Data.output_folder)
Data.Cascade.saveProcessedInferences(Data.output_folder)

Saving Processed Inferences...
Finished Saving Processed Inferences


### Infer Discrete Spike Times

In [16]:
Data.Cascade.inferDiscreteSpikes()

Infer spikes for neuron 1 out of 662
Infer spikes for neuron 2 out of 662
Infer spikes for neuron 3 out of 662
Infer spikes for neuron 4 out of 662
Infer spikes for neuron 5 out of 662
Infer spikes for neuron 6 out of 662
Infer spikes for neuron 7 out of 662
Infer spikes for neuron 8 out of 662
Infer spikes for neuron 9 out of 662
Infer spikes for neuron 10 out of 662
Infer spikes for neuron 11 out of 662
Infer spikes for neuron 12 out of 662
Infer spikes for neuron 13 out of 662
Infer spikes for neuron 14 out of 662
Infer spikes for neuron 15 out of 662
Infer spikes for neuron 16 out of 662
Infer spikes for neuron 17 out of 662
Infer spikes for neuron 18 out of 662
Infer spikes for neuron 19 out of 662
Infer spikes for neuron 20 out of 662
Infer spikes for neuron 21 out of 662
Infer spikes for neuron 22 out of 662
Infer spikes for neuron 23 out of 662
Infer spikes for neuron 24 out of 662
Infer spikes for neuron 25 out of 662
Infer spikes for neuron 26 out of 662
Infer spikes for neur

### Save Progress

In [17]:
Data.Cascade.saveSpikeInference(Data.output_folder)

C:\ProgramData\Anaconda3\envs\Calcium-Imaging-Analysis-Pipeline\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
## Export to MATLab for MATLab Analyses

In [18]:
Data.Cascade.exportSpikeProb(Data.output_folder)
Data.Cascade.exportSpikeInference(Data.output_folder)

C:\Users\YUSTE\AppData\Roaming\Python\Python37\site-packages\scipy\io\matlab\mio5.py:450: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  narr = np.asanyarray(source)


In [19]:
### Visualize Noise Histogram
from AnalysisModules.StaticPlotting import plotNoise
plotNoise(Data.Cascade.traces, Data.Cascade.frame_rate)

In [ ]:
## Visualize Spike Inference
from AnalysisModules.StaticPlotting import plotSpikeInference
plotSpikeInference(Data.Cascade.spike_prob, Data.Cascade.spike_time_estimates, Data.Cascade.traces, Data.Cascade.frame_rate)

In [21]:
from AnalysisModules.StaticPlotting import plotFiringRateMatrix
FiringRate = Data.Cascade.ProcessedInferences.firing_rates.copy()
import sklearn.preprocessing
FiringRate = sklearn.preprocessing.minmax_scale(FiringRate, axis=1)
import scipy.ndimage
FiringRate = scipy.ndimage.gaussian_filter1d(FiringRate, sigma=3, axis=1)
plotFiringRateMatrix(FiringRate, Data.Cascade.frame_rate)

ValueError: The number of FixedLocator locations (70), usually from a call to set_ticks, does not match the number of ticklabels (71).

In [23]:
import matplotlib
matplotlib.use('Qt5Agg')

from matplotlib import pyplot as plt

In [14]:
from cascade2p.utils import plot_dFF_traces
import numpy as np
subset = np.random.randint(Data.Cascade.traces.shape[0], size=10)
plot_dFF_traces(Data.Cascade.traces, subset, frame_rate=9.85, spiking=Data.Cascade.spike_prob)

array([ 0.        ,  0.10152284,  0.20304569,  0.30456853,  0.40609137,
        0.50761421,  0.60913706,  0.7106599 ,  0.81218274,  0.91370558,
        1.01522843,  1.11675127,  1.21827411,  1.31979695,  1.4213198 ,
        1.52284264,  1.62436548,  1.72588832,  1.82741117,  1.92893401,
        2.03045685,  2.1319797 ,  2.23350254,  2.33502538,  2.43654822,
        2.53807107,  2.63959391,  2.74111675,  2.84263959,  2.94416244,
        3.04568528,  3.14720812,  3.24873096,  3.35025381,  3.45177665,
        3.55329949,  3.65482234,  3.75634518,  3.85786802,  3.95939086,
        4.06091371,  4.16243655,  4.26395939,  4.36548223,  4.46700508,
        4.56852792,  4.67005076,  4.7715736 ,  4.87309645,  4.97461929,
        5.07614213,  5.17766497,  5.27918782,  5.38071066,  5.4822335 ,
        5.58375635,  5.68527919,  5.78680203,  5.88832487,  5.98984772,
        6.09137056,  6.1928934 ,  6.29441624,  6.39593909,  6.49746193,
        6.59898477,  6.70050761,  6.80203046,  6.9035533 ,  7.00

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(111)
ax1.plot(Data.Cascade.traces[87, :])
ax1.plot(Data.Cascade.spike_prob[87, :])
plt.show()

KeyboardInterrupt: 

In [26]:
step_size=600
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(111)
sns.heatmap(FiringRate, ax=ax1, cmap="Spectral_r")

<AxesSubplot:>

In [27]:
ax1.set_title("Firing Rate Map")

Text(0.5, 1.0, 'Firing Rate Map')

In [24]:
import seaborn as sns

In [28]:
ax1.set_xticks((range(0, int(FiringRate.shape[1]), int(step_size))),
                   labels=(range(0, int(FiringRate.shape[1]/10),
                                 int(step_size/10))))

In [29]:
ax1.set_yticks([0, int(FiringRate.shape[0])], labels=[0, int(FiringRate.shape[0])])

In [30]:
plt.show()